In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Dagshub Setup

In [ ]:
!pip install mlflow dagshub

In [ ]:
import dagshub
dagshub.auth.add_app_token('ed22b8f88bed7c968c7425f62448b33703f4e094')
dagshub.init(repo_owner='vvaza22', repo_name='ml-assignment-2', mlflow=True)

# Test Dataset

In [ ]:
# Load each data table
transactions = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_transaction.csv")
print(len(transactions))
identities = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_identity.csv")
print(len(identities))

# Merge data tables
df = pd.merge(transactions, identities, on="TransactionID", how="left")

# Free up memory
del transactions
del identities

In [ ]:
df.shape

In [ ]:
# Train set cointains id_01 while test set for some reason contains id-01 columns
cols = list(df.columns)
id_cols = [col for col in cols if col.startswith("id")]

replace_cols = {}
for col in id_cols:
    replace_cols[col] = col.replace("id-", "id_")
replace_cols

In [ ]:
df = df.rename(columns=replace_cols)

# Submission

In [ ]:
!pip install --upgrade scikit-learn==1.3.2 imbalanced-learn==0.11.0

In [ ]:
import mlflow
logged_model = 'runs:/00ad88b45697450ebb88d8d1867f1734/XGBoost'
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [ ]:
pred = loaded_model.predict(df)
pred

In [ ]:
submission = pd.DataFrame({
    'TransactionID': df['TransactionID'],
    'isFraud': pred,
})

submission.to_csv('submission.csv', index=False)